In [2]:
import os
import sys
import dill
import numpy as np
import pandas as pd
import multiprocessing as mp
from pathlib import Path
from pyspi.calculator import Calculator
from pyspi.data import Data
import logging
from datetime import datetime
from tqdm import tqdm


In [3]:
npy_file = Path('/media/dan/Data/git/ubiquitous-spork/common_average_reref/two_epochs/run_data/test_data.npy')

CONFIG_FILE = Path('/media/dan/Data/git/ubiquitous-spork/common_average_reref/two_epochs/test.yaml')
base_name = Path(npy_file).stem
data = Data(data=str(npy_file), dim_order='ps', name=base_name, normalise=True)

# Create calculator with config file
calc = Calculator(configfile=str(CONFIG_FILE))
calc.load_dataset(data)
calc.compute()

[1/2] Skipping detrending of time series in the dataset...
[2/2] Normalising (z-scoring) each time series in the dataset...

Checking if optional dependencies exist...
Starting JVM with java class /home/dan/anaconda3/envs/networks/lib/python3.12/site-packages/pyspi/lib/jidt/infodynamics.jar.
Loading configuration file: /media/dan/Data/git/ubiquitous-spork/common_average_reref/two_epochs/test.yaml
*** Importing module .statistics.spectral
[0] Adding SPI .statistics.spectral.DirectedTransferFunction(x,y,{'fmin': 0.00048828125, 'fmax': 0.001953125})
Succesfully initialised SPI with identifier "dtf_multitaper_mean_fs-1_fmin-0-000488_fmax-0-00195" and labels ['unsigned', 'spectral', 'directed', 'lagged']
[1] Adding SPI .statistics.spectral.DirectedTransferFunction(x,y,{'fmin': 0.001953125, 'fmax': 0.00390625})
Succesfully initialised SPI with identifier "dtf_multitaper_mean_fs-1_fmin-0-00195_fmax-0-00391" and labels ['unsigned', 'spectral', 'directed', 'lagged']
[2] Adding SPI .statistics.s

Processing [None: dtf_multitaper_mean_fs-1_fmin-0-000488_fmax-0-00195]:   0%|          | 0/160 [00:00<?, ?it/s]Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Processing [None: dcoh_multitaper_mean_fs-1_fmin-0-000488_fmax-0-00195]:   1%|          | 1/160 [00:01<03:48,  1.44s/it]Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Processing [None: pdcoh_multitaper_mean_fs-1_fmin-0-000488_fmax-0-00195]:  13%|█▎        | 21/160 [00:03<00:20,  6.65it/s]Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Processing [None:


Calculation complete. Time taken: 10.1381s

SPI Computation Results Summary

Total number of SPIs attempted: 160
Number of SPIs successfully computed: 136 (85.00%)
------------------------------------------------------------
Category       | Count | Percentage
------------------------------------------------------------
Successful     |   136 |  85.00%
NaNs           |     0 |   0.00%
Partial NaNs   |    24 |  15.00%
------------------------------------------------------------

[24] SPIs which produced partial NaN outputs:
------------------------------------------------------------
1. sgc_nonparametric_mean_fs-1_fmin-0-0342_fmax-0-0732
2. sgc_nonparametric_mean_fs-1_fmin-0-000488_fmax-0-0342
3. sgc_nonparametric_max_fs-1_fmin-0-0342_fmax-0-0732
4. sgc_nonparametric_max_fs-1_fmin-0-000488_fmax-0-0342
5. sgc_parametric_mean_fs-1_fmin-0-000488_fmax-0-00195_order-20
6. sgc_parametric_mean_fs-1_fmin-0-00195_fmax-0-00391_order-20
7. sgc_parametric_mean_fs-1_fmin-0-00391_fmax-0-00586_order-

In [5]:
calc.table['sgc_parametric_max_fs-1_fmin-0-0732_fmax-0-122_order-20']

process,proc-0,proc-1,proc-2
proc-0,NaN,NaN,3.032472
proc-1,NaN,NaN,NaN
proc-2,0.75069,-0.144591,NaN


In [9]:
"""
granger_diag.py
---------------

Quick diagnostics for Granger-causality preprocessing on an array `z`
with shape (channels, time).

Checks performed
----------------
1. Demean / detrend status for each channel.
2. Sample-size adequacy for a chosen VAR order p.
3. Whiteness of residuals from a VAR(p) model (Ljung–Box test).
4. Stability of the VAR: all eigenvalues of A(z) inside the unit circle.

Dependencies: numpy, scipy, statsmodels (<pip install numpy scipy statsmodels>)
"""

import numpy as np
from scipy import signal
from statsmodels.tsa.api import VAR
from statsmodels.stats.diagnostic import acorr_ljungbox


def check_demean_trend(z, tol_mean=1e-3, tol_slope=1e-3):
    """
    Returns boolean masks indicating channels that deviate from
    zero mean or exhibit a linear trend.

    Parameters
    ----------
    z : ndarray, shape (channels, time)
    tol_mean : float
        Absolute mean > tol_mean → flagged as not demeaned.
    tol_slope : float
        |slope| > tol_slope → flagged as trending.

    Returns
    -------
    bad_mean : ndarray(bool), shape (channels,)
    bad_trend : ndarray(bool), shape (channels,)
    """
    ch, t = z.shape
    time = np.arange(t)
    # mean check
    bad_mean = np.abs(z.mean(axis=1)) > tol_mean

    # linear trend check (simple least-squares slope)
    xm = time.mean()
    denom = np.sum((time - xm) ** 2)
    slopes = np.sum((time - xm) * (z - z.mean(axis=1, keepdims=True)), axis=1) / denom
    bad_trend = np.abs(slopes) > tol_slope
    return bad_mean, bad_trend


def check_sample_size(n_channels, n_time, p, factor=10):
    """
    Returns True if n_time >= factor * (#parameters for VAR(p)), else False.
    """
    n_params = n_channels ** 2 * p
    print(n_params)
    return n_time >= factor * n_params, n_params


from statsmodels.tsa.api import VAR
from statsmodels.stats.diagnostic import acorr_ljungbox
import numpy as np

def whiteness_and_stability(z, p, alpha=0.05, lags_whiteness=None):
    """
    Fits VAR(p) and returns:
      * ljung_pvals : array of Ljung–Box p-values [channels, lags]
      * stable      : True if all eigenvalues inside unit circle
      * results     : statsmodels VARResults
    """
    # statsmodels expects (time, channels)
    model = VAR(z.T)
    results = model.fit(p)
    resid = results.resid  # (time-p, channels)

    # Choose a reasonable # of lags for Ljung–Box
    if lags_whiteness is None:
        lags_whiteness = max(1, min(20, resid.shape[0] // 5))

    ljung_pvals = np.empty((resid.shape[1], lags_whiteness))

    for k in range(resid.shape[1]):
        # Always request a DataFrame to avoid tuple/DataFrame ambiguity across versions
        df = acorr_ljungbox(resid[:, k], lags=lags_whiteness, return_df=True)
        # Extract p-values as a NumPy array
        ljung_pvals[k, :] = df["lb_pvalue"].to_numpy()

    # Stability: all roots inside the unit circle
    stable = np.all(np.abs(results.roots) < 1.0)
    return ljung_pvals, stable, results


def run_diagnostics(z, p=20, tol_mean=1e-3, tol_slope=1e-3,
                    factor=10, alpha=0.05):
    """
    Wrapper that prints a concise report.
    """
    n_channels, n_time = z.shape
    print("=" * 60)
    print(f"Input array: {n_channels} channels × {n_time} samples")
    print(f"Checking VAR order p = {p}\n")

    # 1. Demean / detrend
    bad_mean, bad_trend = check_demean_trend(z, tol_mean, tol_slope)
    if bad_mean.any() or bad_trend.any():
        bad_idx = np.where(bad_mean | bad_trend)[0]
        print(f"⚠️  {bad_idx.size}/{n_channels} channels are not "
              f"demeaned/trend-free:", bad_idx)
    else:
        print("✅ All channels appear demeaned and trend-free "
              f"(tol_mean={tol_mean}, tol_slope={tol_slope})")

    # 2. Sample-size adequacy
    ok_ss, n_params = check_sample_size(n_channels, n_time, p, factor)
    print(f"\nRequired samples (factor {factor}×params): "
          f"{factor*n_params}, available: {n_time}")
    print("✅" if ok_ss else "⚠️ ", "Sample size check")

    # 3 & 4. Fit VAR, whiteness, stability
    ljung_pvals, stable, results = whiteness_and_stability(z, p, alpha)
    white_mask = (ljung_pvals > alpha).all(axis=1)
    n_white = white_mask.sum()
    print(f"\nLjung–Box α={alpha} → {n_white}/{n_channels} channels "
          f"have white residuals at all lags "
          f"(lags tested ≤ {ljung_pvals.shape[1]})")

    print("✅" if stable else "⚠️ ", "VAR stability "
          f"({np.sum(np.abs(results.roots) >= 1)} eigenvalues "
          f"outside unit circle)")

    print("=" * 60)
    return {
        "bad_mean": bad_mean,
        "bad_trend": bad_trend,
        "sample_size_ok": ok_ss,
        "ljung_pvals": ljung_pvals,
        "white_mask": white_mask,
        "stable": stable,
        "results": results,
    }


# ---------------------------------------------------------------------
# Example usage
# ---------------------------------------------------------------------
# dummy example: 5 channels × 2048 samples, AR(2) process


rng = np.random.default_rng(42)

# load npy file
z = np.load('/media/dan/Data/git/ubiquitous-spork/common_average_reref/two_epochs/run_data/test_data.npy')

# run diagnostics
run_diagnostics(z, p=20)

Input array: 3 channels × 1024 samples
Checking VAR order p = 20

✅ All channels appear demeaned and trend-free (tol_mean=0.001, tol_slope=0.001)
180

Required samples (factor 10×params): 1800, available: 1024
⚠️  Sample size check

Ljung–Box α=0.05 → 0/3 channels have white residuals at all lags (lags tested ≤ 20)
⚠️  VAR stability (60 eigenvalues outside unit circle)


{'bad_mean': array([False, False, False]),
 'bad_trend': array([False, False, False]),
 'sample_size_ok': False,
 'ljung_pvals': array([[1.88813728e-01, 4.11871059e-01, 2.36498128e-01, 8.99087822e-04,
         1.97750364e-03, 1.43524057e-03, 1.41739608e-07, 2.14445880e-07,
         5.25182503e-07, 3.51019584e-10, 1.36732730e-10, 3.98310974e-11,
         1.38221953e-11, 3.15585392e-11, 1.31521754e-12, 1.76857826e-12,
         2.94878898e-12, 2.64205350e-14, 3.74923833e-14, 9.08414017e-14],
        [3.41970509e-01, 6.26524282e-01, 1.82446748e-01, 3.38286995e-02,
         6.39850790e-02, 3.41292189e-03, 4.40988444e-08, 5.55427490e-08,
         1.39046010e-07, 6.62821222e-11, 9.19237509e-12, 6.29191563e-12,
         4.87967676e-12, 1.21813907e-11, 5.21798227e-12, 9.18274459e-12,
         1.01472852e-11, 1.81207816e-13, 1.66206111e-13, 2.89749774e-13],
        [4.00175526e-01, 6.58018667e-01, 2.16515580e-01, 7.12245017e-02,
         1.23247092e-01, 1.88982744e-03, 3.99017611e-08, 1.10432297